This tutorial shows a basic run of the Procrustes subspace adaptation (PSA) algorithm. PSA 
We begin by importing all dependencies:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils
from utils import NeuralNetwork
from utils import train_model
import psa

import torch 
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import StandardScaler

# Multi